# Build StudentEval

This notebook builds the StudentEval dataset and pushes it to the Hugging Face Hub.
You need access to the not-fully-anonymized data (`interactions.csv`) to run this notebook.
You also need `problems.yaml`.

In [1]:
import datasets
import pandas as pd
import yaml
from pathlib import Path

Load the interactions, drop the `id` (which identifies a database row) and replace pseudonyms.

In [2]:
interactions_df = pd.read_csv("../raw_data/interactions.csv", index_col=0)
interactions_df = interactions_df.drop(columns=["id"])
# Replace pseudonyms with student0, student1, ...
username_map = dict((old, f"student{ix}") for (ix, old) in  enumerate(interactions_df["username"].unique()))
interactions_df["username"] = interactions_df["username"].apply(lambda x: username_map[x])
interactions_df.head()

,username,problem,submitted_text,tests_passed,total_tests,prompt,completion,first_attempt,last_attempt,is_success,is_first_success,is_last_success,is_first_failure,is_last_failure
1,student0,remove_odd,takes in a list of numbers. returns the list o...,1,3,"def remove_odd(lst):\n """"""\n takes in a ...",return [x for x in lst if x % 2 == 0]\n\nprint...,True,False,False,False,False,True,False
2,student0,remove_odd,takes in a list of numbers. returns the list o...,1,3,"def remove_odd(lst):\n """"""\n takes in a ...",return [num for num in lst if num % 2 == 0]\n,False,False,False,False,False,False,False
3,student0,remove_odd,takes in a list of numbers. returns the list o...,1,3,"def remove_odd(lst):\n """"""\n takes in a ...",return [num for num in lst if num % 2 == 0]\n,False,False,False,False,False,False,False
10,student0,reduce,takes in a string and returns a string with '^...,3,3,"def reduce(val):\n """"""\n takes in a stri...","return val.replace('^', '_')\n\n",True,True,True,True,False,False,False
11,student0,convert,takes a list of numbers and returns a list wit...,0,3,"def convert(lst):\n """"""\n takes a list o...",new_lst = []\n for i in lst:\n if i ...,True,False,False,False,False,True,False


Load the problems and assemble both tests and prints.

In [3]:
with Path("../../charlie-the-coding-cow/cli/problems.yaml").open() as f:
        problems = yaml.safe_load(f)

tests_df = [ ] 
for (key, value) in problems.items():
    signature = value["signature"].lstrip()
    entrypoint = signature[4:].split("(")[0]
    tests = [ (test["input"].strip(), test["output"].strip()) for test in value["tests"] ]
    assertions = "\n".join(f"assert {entrypoint}({input}) == {output}" for (input, output) in tests)
    tests_df.append({ 
        "problem": key,
        "entrypoint": entrypoint,
        "assertions": assertions,
        "prints": "\n".join(f"print({entrypoint}({input}))" for (input, _) in tests)
    })
tests_df = pd.DataFrame(tests_df)
tests_df.head()


,problem,entrypoint,assertions,prints
0,shorten_url,shorten_url,assert shorten_url('https://a') == 'a'\nassert...,print(shorten_url('https://a'))\nprint(shorten...
1,apply_operations,apply_operations,"assert apply_operations([ 1, 2, 3 ], [ '+', '*...","print(apply_operations([ 1, 2, 3 ], [ '+', '*'..."
2,add_up,add_up,"assert add_up([ [ 'one', 2, 3, 4 ] ]) == 9\nas...","print(add_up([ [ 'one', 2, 3, 4 ] ]))\nprint(a..."
3,subtract_add,subtract_add,"assert subtract_add(['1', '2', '3', '4']) == -...","print(subtract_add(['1', '2', '3', '4']))\npri..."
4,convert,convert,"assert convert([0, 1, 2, 3]) == ['ABCD']\nasse...","print(convert([0, 1, 2, 3]))\nprint(convert([0..."


In [4]:
joined = tests_df.merge(interactions_df, on=["problem"], how="inner")
joined.head()


,problem,entrypoint,assertions,prints,username,submitted_text,tests_passed,total_tests,prompt,completion,first_attempt,last_attempt,is_success,is_first_success,is_last_success,is_first_failure,is_last_failure
0,add_up,add_up,"assert add_up([ [ 'one', 2, 3, 4 ] ]) == 9\nas...","print(add_up([ [ 'one', 2, 3, 4 ] ]))\nprint(a...",student1,"takes a list of strings, integers, and floats ...",1,3,"def add_up(arr):\n """"""\n takes a list of...","return sum([x for x in arr if isinstance(x, (i...",True,False,False,False,False,True,False
1,add_up,add_up,"assert add_up([ [ 'one', 2, 3, 4 ] ]) == 9\nas...","print(add_up([ [ 'one', 2, 3, 4 ] ]))\nprint(a...",student1,"takes a list of strings, integers, and floats ...",0,3,"def add_up(arr):\n """"""\n takes a list of...",return sum(arr)\n,False,False,False,False,False,False,False
2,add_up,add_up,"assert add_up([ [ 'one', 2, 3, 4 ] ]) == 9\nas...","print(add_up([ [ 'one', 2, 3, 4 ] ]))\nprint(a...",student1,"takes a list of strings, integers, and floats ...",1,3,"def add_up(arr):\n """"""\n takes a list of...",sum = 0\n for i in arr:\n if type(i)...,False,True,False,False,False,False,True
3,add_up,add_up,"assert add_up([ [ 'one', 2, 3, 4 ] ]) == 9\nas...","print(add_up([ [ 'one', 2, 3, 4 ] ]))\nprint(a...",student6,"Input: list of items, that may be a combinatio...",1,3,"def add_up(arr):\n """"""\n Input: list of ...",total = 0\n for item in arr:\n if ty...,True,False,False,False,False,True,False
4,add_up,add_up,"assert add_up([ [ 'one', 2, 3, 4 ] ]) == 9\nas...","print(add_up([ [ 'one', 2, 3, 4 ] ]))\nprint(a...",student6,"Input: list of items, that may be a combinatio...",1,3,"def add_up(arr):\n """"""\n Input: list of ...",new_list = []\n for item in arr:\n i...,False,False,False,False,False,False,False


In [5]:
item = joined.iloc[100]
print(item["prompt"] + item["completion"] + item["assertions"])
print("*" * 80)
print(item["prompt"] + item["completion"] + item["prints"])


def subtract_add(lst):
    """
    Change the first two items into integers Subtract the second item in the array list from the first item. change the third and forth items into integers then subtract the forth item from the third item then add the two sums. if there are just two items in list change those two items into integers subtract the second one from the first one and return the product. if there are no items then return zero
    """
    if len(lst) == 0:
        return 0
    elif len(lst) == 2:
        return int(lst[0]) - int(lst[1])
    else:
        return int(lst[0]) - int(lst[1]) + int(lst[2]) - int(lst[3])
assert subtract_add(['1', '2', '3', '4']) == -2
assert subtract_add(['0', '2']) == -2
assert subtract_add(['0', '0']) == 0
assert subtract_add([ ]) == 0
********************************************************************************
def subtract_add(lst):
    """
    Change the first two items into integers Subtract the second item in the array list from the first ite

In [6]:
len(joined)

1749

In [7]:
studenteval = datasets.DatasetDict(
    { "test": datasets.Dataset.from_pandas(joined) 
})

In [8]:
studenteval.push_to_hub("nuprl/StudentEval", private=True)

Pushing split test to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]